# Analyzing Flux Perturbations for Objective I #

### File to generate Fig5, Fig 6, Fig S11 ###

In [1]:
import sys
import colorlover as cl
from plotly import tools, subplots
import plotly.colors as pycolors
import plotly.graph_objs as go
import cameo
from cameo.flux_analysis.simulation import FluxDistributionResult
import pandas as pd
import os
import colorlover as cl
import plotly.io as pio
import pickle

import numpy as np
from copy import deepcopy
pio.templates.default=None
try:
    _ = __IPYTHON__
except NameError:
    from plotly.offline import plot
else:
    if 'ipykernel' in sys.modules:
        from plotly.offline import init_notebook_mode
        from plotly.offline import iplot as plot
        from IPython.display import HTML
        HTML("""
             <script>
              var waitForPlotly = setInterval( function() {
              if( typeof(window.Plotly) !== "undefined" ){
              MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
              MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
              clearInterval(waitForPlotly);}}, 250 );
            </script>
            """
        )
        init_notebook_mode(connected=True)
    elif 'IPython' in sys.modules:
        from plotly.offline import plot
    else:
        warn('Unknown ipython configuration')
        from plotly.offline import plot

In [2]:
pd.options.display.max_rows=600
pd.options.display.max_columns=150
wt_model = cameo.models.bigg.iJO1366
model = cameo.models.bigg.iJO1366
relevant_reactions = [reaction.id for reaction in wt_model.reactions 
                      if reaction.subsystem not in ['Transport, Outer Membrane',
                                                    'Transport, Inner Membrane',
                                                    'Inorganic Ion Transport and Metabolism',
                                                    'Transport, Outer Membrane Porin',
                                                    'Intracellular demand',
                                                    'Extracellular exchange']]
relevant_reactions_set = set(relevant_reactions)
metabolite_names_dict = {'Spermidine': 'Spermidine',
                         'L-Tryptophan': 'L-Tryptophan',
                         'Thymidine C10H14N2O5': 'Thymidine',
                         'L-Phenylalanine': 'L-Phenylalanine',
                         'Adenosine': 'Adenosine',
                         'Indole': 'Indole',
                         'L-Tyrosine': 'L-Tyrosine',
                         'Inosine': 'Inosine',
                         'Xanthosine': 'Xanthosine',
                         'Cytidine': 'Cytidine',
                         'Uridine': 'Uridine',
                         'Quinate': 'Quinate',
                         'L-Isoleucine': 'L-Isoleucine',
                         'L-Lysine': 'L-Lysine',
                         'Hexanoate (n-C6:0)': 'Hexanoate',
                         'L-Leucine': 'L-Leucine',
                         'L-Arginine': 'L-Arginine',
                         'L-Histidine': 'L-Histidine',
                         'D-Gluconate': 'D-Gluconic Acid',
                         'L-Idonate': 'L-Idonic Acid',
                         '5-Dehydro-D-gluconate': '5-Ketogluconate',
                         'Citrate': 'Citric Acid',
                         'Agmatine': 'Agmatine',
                         'Ornithine': 'Ornithine',
                         'L-Proline': 'L-Proline',
                         'L-Valine': 'L-Valine',
                         'Thymine C5H6N2O2': 'Thymine',
                         'Adenine': 'Adenine',
                         'Guanine': 'Guanine',
                         'Hypoxanthine': 'Hypoxanthine',
                         'O-Acetyl-L-serine': 'O-Acetyl-L-serine',
                         'Xanthine': 'Xanthine',
                         'L-Glutamate': 'L-Glutamate',
                         '2-Oxoglutarate': '\u03b1-Ketoglutarate',
                         'Putrescine': 'Putrescine',
                         'L-Threonine': 'L-Threonine',
                         '4-Aminobutanoate': '\u03b3-Aminobutyrate',
                         'L-Homoserine': 'L-Homoserine',
                         'Allantoin': 'Allantoin',
                         'Succinate': 'Succinate',
                         'Uracil': 'Uracil',
                         'L-Asparagine': 'L-Asparagine',
                         'L-Malate': 'L-Malate',
                         'L-Aspartate': 'L-Aspartate',
                         'L-Cysteine': 'L-Cysteine',
                         'Glycerol 3-phosphate': 'Glycerol3-phosphate',
                         '3-Hydroxypropanoate': '3-Hydroxypropanoate',
                         '(S)-Propane-1,2-diol': '(S)-Propanediol',
                         '(R)-Propane-1,2-diol': '(R)-Propanediol',
                         'D-Alanine': 'D-Alanine',
                         'Glycerol': 'Glycerol',
                         '(R)-Glycerate': '(R)-Glycerate',
                         'D-Glyceraldehyde': 'Glyceraldehyde',
                         'L-Lactate': 'L-Lactate',
                         'Dihydroxyacetone': 'Dihydroxyacetone',
                         'L-Alanine': 'L-Alanine',
                         'D-Lactate': 'D-Lactate',
                         'L-Serine': 'L-Serine',
                         'Pyruvate': 'Pyruvate',
                         'Ethanolamine': 'Ethanolamine',
                         'Ethanol': 'Ethanol',
                         'Acetaldehyde': 'Acetaldehyde',
                         'Glycine': 'Glycine',
                         'Acetate': 'Acetate',
                         'Glycolate C2H3O3': 'Glycolate',
                         'Urea CH4N2O': 'Urea',
                         'Formate': 'Formate',
                         'Reduced glutathione': 'Glutathione',
                         '5-Methylthio-D-ribose': '5-Methylthioribose',
                         '1,5-Diaminopentane': 'Cadaverine'}

metabolite_lookup_dict = {metabolite_names_dict[key]:key for key in metabolite_names_dict}

carbon_dict = {metabolite.name:str(metabolite.elements['C'])
               if 'C' in metabolite.elements else '0'
               for metabolite in model.metabolites}

carbon_dict = {metabolite:carbon_dict[metabolite] 
               if int(carbon_dict[metabolite])<=6 else '>6'
               for metabolite in metabolite_names_dict}

carbon_dict = {metabolite_names_dict[metabolite]:carbon_dict[metabolite] for metabolite in carbon_dict}

data = {'names':[product for product in carbon_dict.keys()], 'number':list(carbon_dict.values())}
metabolite_df = pd.DataFrame(data,index=list(carbon_dict.keys()))
metabolite_order = metabolite_df.sort_values(by=['number','names']).names.values

aa_metabolism_subsystems=['Alanine and Aspartate Metabolism',
                          'Arginine and Proline Metabolism',
                          'Cysteine Metabolism',
                          'Glutamate Metabolism',
                          'Glycine and Serine Metabolism',
                          'Histidine Metabolism',
                          'Methionine Metabolism',
                          'Threonine and Lysine Metabolism',
                          'Tyrosine, Tryptophan, and Phenylalanine Metabolism',
                          'Valine, Leucine, and Isoleucine Metabolism']
ijo_sub = {reaction.id:reaction.subsystem if reaction.subsystem not in aa_metabolism_subsystems
                                                    else 'Amino Acid Metabolism'
           for reaction in wt_model.reactions}


Academic license - for non-commercial use only


# Import Preprocessed FVA #

In [3]:
pickle_off_1 = open("fva_final_prod.pickle","rb")
fva = pickle.load(pickle_off_1)
pickle_off_1.close()

pickle_off_1 = open("parsed_fva_final_prod.pickle","rb")
parsed_fva = pickle.load(pickle_off_1)
pickle_off_1.close()

# WT FVA, PFBA, Preprocessing #

In [4]:
wt_model = cameo.models.bigg.iJO1366
wt_model.solver='gurobi'

wt_df = cameo.flux_variability_analysis(wt_model,fraction_of_optimum=1,reactions=relevant_reactions).data_frame
wt_pfba = cameo.pfba(wt_model).data_frame.reindex(relevant_reactions)
wt_df = pd.concat([wt_pfba,wt_df], axis=1, sort=True)
del(wt_pfba)
wt_df.columns = ['pfba','lb','ub']
min_biomass = min(parsed_fva['inflexible']['pfba'].loc['BIOMASS_Ec_iJO1366_core_53p95M'])
cutoff = 0.001*wt_df['pfba'].loc['BIOMASS_Ec_iJO1366_core_53p95M']/min_biomass
wt_df['pfba'][abs(wt_df['pfba'])<=cutoff]=0
wt_df['lb'][abs(wt_df['lb'])<=cutoff]=0
wt_df['ub'][abs(wt_df['ub'])<=cutoff]=0
wt_df['range'] = wt_df['ub'] - wt_df['lb']
wt_df['range'][abs(wt_df['range'])<=cutoff]=0

prod_pfba = parsed_fva['inflexible']['pfba'][metabolite_order]

wt_used_reactions = set(wt_df[wt_df['pfba']!=0].index)
wt_unused_reactions = relevant_reactions_set - wt_used_reactions

# Flux changes for each product#

In [5]:
change_df = prod_pfba.reindex(wt_used_reactions).abs().divide(wt_df['pfba'].reindex(wt_used_reactions).abs(),axis=0).round(3)

on_reactions = wt_unused_reactions.intersection(parsed_fva['used_reactions_set'])
off_reactions = set(prod_pfba.reindex(wt_used_reactions)[(prod_pfba.reindex(wt_used_reactions)==0).any(axis=1)].index)
significantly_upregulated_reactions = set(change_df[change_df[change_df>1.1].count(axis=1)>0].index)
significantly_upregulated_reactions_boolean = change_df>1.1
significantly_downregulated_reactions = set(change_df[change_df[(change_df<change_df.loc['BIOMASS_Ec_iJO1366_core_53p95M']*0.9)&(change_df>0)].count(axis=1)!=0].index)
significantly_downregulated_reactions_boolean = (change_df<change_df.loc['BIOMASS_Ec_iJO1366_core_53p95M']*0.9)&(change_df>0)
print("Total No. of unique reactions switched on: ", len(on_reactions))
print("Total No. of unique reactions switched off:", len(off_reactions))
print("Total No. of unique reactions significantly upregulated:", len(significantly_upregulated_reactions))
print("Total No. of unique reactions significantly downregulated:", len(significantly_downregulated_reactions))


on_df = deepcopy(prod_pfba).reindex(on_reactions).abs()
on_df[on_df==0] = np.nan

off_df = deepcopy(prod_pfba).reindex(off_reactions)
off_df[off_df!=0] = np.nan
off_df = (off_df.transpose() + wt_df['pfba'].reindex(off_reactions)).transpose().abs()

upregulated_df = (prod_pfba.reindex(wt_used_reactions)[change_df>1].abs().transpose() -\
                  wt_df['pfba'].reindex(wt_used_reactions).abs().transpose()).transpose()
upregulated_df = upregulated_df.drop(['BIOMASS_Ec_iJO1366_core_53p95M','ATPM'])
significantly_upregulated_df = upregulated_df.loc[significantly_upregulated_reactions][significantly_upregulated_reactions_boolean]

downregulated_df = (wt_df['pfba'].reindex(wt_used_reactions).abs().transpose() - \
                    prod_pfba.reindex(wt_used_reactions)[((change_df<1)&(change_df>0))].abs().transpose()).transpose()
downregulated_df = downregulated_df.drop(['BIOMASS_Ec_iJO1366_core_53p95M','ATPM'])

significantly_downregulated_df = downregulated_df.loc[significantly_downregulated_reactions][significantly_downregulated_reactions_boolean]


Total No. of unique reactions switched on:  111
Total No. of unique reactions switched off: 16
Total No. of unique reactions significantly upregulated: 183
Total No. of unique reactions significantly downregulated: 41


In [9]:
on_trace = go.Bar(x=metabolite_order, y=on_df.count().values,
                  name='Number of Reactions Switched On', 
                  marker=dict(color='#4DAF4A', line=dict(width=0, color='black')),
                  legendgroup='Switched On', showlegend=True, orientation='v', width=0.9, opacity=0.75)
off_trace = go.Bar(x=metabolite_order, y=off_df.count().values,
                   name='Number of Reactions Switched Off', marker=dict(color='#E41A1C', line=dict(width=0, color='black')),
                   legendgroup='Switched Off', showlegend=True, orientation='v', width=0.9, opacity=0.75)
on_flux_trace = go.Scatter(x=metabolite_order,y=on_df.sum().values,name='Sum of Fluxes Switched On',yaxis='y2',mode='markers',
                           marker_color=cl.scales['5']['qual']['Set1'][2], marker=dict(size=8, symbol='triangle-up',
                                                                                       line=dict(width=0.5, color='black')))
off_flux_trace = go.Scatter(x=metabolite_order,y=off_df.sum().values,name='Sum of Fluxes Switched Off',yaxis='y2',mode='markers',
                            marker_color=cl.scales['5']['qual']['Set1'][0], marker=dict(size=8, symbol='triangle-down',
                                                                                        line=dict(width=0.5, color='black')))

layout = go.Layout(height=425, width=950, barmode='stack', margin=dict(l=50, t=50), 
                   showlegend=True, legend=dict(orientation='h', x=0, y=-0.5, font=dict(size=11.38, color='black')),
                   xaxis=dict(showline=True, linewidth=1.5, linecolor='black', mirror=True, ticks='outside', ticklen=3, 
                              tickfont=dict(size=9.75, family='Arial', color='black'), tickangle=-90, side='bottom', 
                              range=(-0.5,69.5)),
                   yaxis=dict(title='Number of Reactions', titlefont=dict(family='Arial', size=13, color='black'),
                              showline=True, linewidth=1.5, linecolor='black', ticks='outside', range=(0,35), 
                              tickcolor='black', gridcolor='black', tickfont=dict(size=11.38, color='black'),mirror=False),
                  yaxis2=dict(title='Sum Absolute Flux Change (mmol/gdw.h)', anchor='x', overlaying='y', linewidth=1.5, linecolor='black',
                              titlefont=dict(family='Arial', size=13, color='black'), showline=True, ticks='outside',
                              range=(0,175), tickfont=dict(size=11.38, color='black'), gridcolor='black', mirror=False,
                              side='right'))
fig_on_off = go.Figure(data=[on_trace,off_trace,off_flux_trace, on_flux_trace],layout=layout)


trace_list=[]
upreg_trace = go.Bar(x=metabolite_order, y=significantly_upregulated_df.count().values,
                     name='Number of Significantly Upregulated Reactions', marker_color=cl.scales['5']['qual']['Set1'][1],
                     legendgroup='Number of Significantly Upregulated Reactions', 
                     showlegend=True, orientation='v', width=0.8, opacity=0.75)

downreg_trace = go.Bar(x=metabolite_order, y=significantly_downregulated_df.count().values,
                       name='Number of Significantly Downregulated Reactions', marker_color=cl.scales['5']['qual']['Set1'][4],
                       legendgroup='Number of Significantly Downregulated Reactions', 
                       showlegend=True, orientation='v', width=0.8, opacity=0.75)

upreg_flux_trace = go.Scatter(x=metabolite_order, y=significantly_upregulated_df.sum().values,
                              name='Sum of Upregulated Fluxes',yaxis='y2',mode='markers',
                              marker_color=cl.scales['5']['qual']['Set1'][1], 
                              marker=dict(size=8, symbol='triangle-up', line=dict(width=0.5, color='black')))

downreg_flux_trace = go.Scatter(x=metabolite_order, y=significantly_downregulated_df.sum().values,
                                name='Sum of Downregulated Fluxes',yaxis='y2',mode='markers',
                                marker_color=cl.scales['5']['qual']['Set1'][4], 
                                marker=dict(size=8, symbol='triangle-down', line=dict(width=0.5, color='black')))


layout = go.Layout(height=425, width=950, barmode='stack', margin=dict(l=50, t=50), 
                   showlegend=True, legend=dict(orientation='h', x=0, y=-0.5, font=dict(size=11.38, color='black')),
                   xaxis=dict(showline=True, linewidth=1.5, linecolor='black', mirror=True, ticks='outside', ticklen=3, 
                              tickfont=dict(size=9.75, family='Arial', color='black'), tickangle=-90, side='bottom', 
                              range=(-0.5,69.5)),
                   yaxis=dict(title='Number of Reactions', titlefont=dict(family='Arial', size=13, color='black'),
                              showline=True, linewidth=1.5, linecolor='black', ticks='outside', range=(0,45), 
                              tickcolor='black', gridcolor='black', tickfont=dict(size=11.38, color='black'),mirror=False),
                  yaxis2=dict(title='Sum Absolute Flux Change (mmol/gdw.h)', anchor='x', overlaying='y', linewidth=1.5, linecolor='black',
                              titlefont=dict(family='Arial', size=13, color='black'), showline=True, ticks='outside',
                              range=(0,450), tickfont=dict(size=11.38, color='black'), gridcolor='black', mirror=False,
                              side='right'))

fig_upregulated_downregulated = go.Figure(data=[upreg_trace,downreg_trace,upreg_flux_trace,downreg_flux_trace],layout=layout)






plot(fig_on_off)
plot(fig_upregulated_downregulated)

if not os.path.exists('images'):
    os.mkdir('images')

pio.write_image(fig_on_off, 'images/no_fluxes_productwise_on_off_objA.svg')
pio.write_image(fig_upregulated_downregulated, 'images/no_fluxes_productwise_upreg_downreg_objA.svg')


# Reaction Occurence #

In [6]:
subsystems = ['Glycolysis/Gluconeogenesis',
              'Citric Acid Cycle',
              'Oxidative Phosphorylation',
              'Pentose Phosphate Pathway',
              'Anaplerotic Reactions',
              'Purine and Pyrimidine Biosynthesis',
              'Nucleotide Salvage Pathway',
              'Alternate Carbon Metabolism',
              'Amino Acid Metabolism',
              'Pyruvate Metabolism',
              'Folate Metabolism']


on_reactions_list = list(on_df.count(axis=1).sort_values(ascending=False)[:10].index)
off_reactions_list = list(off_df.count(axis=1).sort_values(ascending=False)[:10].index)
upregulated_reactions_list = list(significantly_upregulated_df.count(axis=1).sort_values(ascending=False)[:10].index)
downregulated_reactions_list = list(significantly_downregulated_df.count(axis=1).sort_values(ascending=False)[:10].index)

important_reactions_set = set(on_reactions_list+off_reactions_list+upregulated_reactions_list+downregulated_reactions_list)
important_reactions_subsystems = {reaction:ijo_sub[reaction] for reaction in important_reactions_set}

del(on_reactions_list, off_reactions_list, upregulated_reactions_list, downregulated_reactions_list, important_reactions_set)
on_count_df = pd.DataFrame(on_df.count(axis=1).sort_values(ascending=False))
on_count_df.columns = ['number']
on_count_df['subsystem'] = list(on_count_df.index.map(ijo_sub))
on_count_df['mean'] = (on_df.abs().mean(axis=1))
on_count_df['std'] = (on_df.abs().std(axis=1))
on_count_df = on_count_df.sort_values(by=['number'],ascending=False)[['number','subsystem','mean','std']]

off_count_df = pd.DataFrame(off_df.count(axis=1).sort_values(ascending=False))
off_count_df.columns = ['number']
off_count_df['subsystem'] = list(off_count_df.index.map(ijo_sub))
off_count_df['mean'] = (off_df.abs().mean(axis=1))
off_count_df['std'] = (off_df.abs().std(axis=1))
off_count_df = off_count_df.sort_values(by=['number'],ascending=False)[['number','subsystem','mean','std']]

upregulated_count_df = pd.DataFrame(significantly_upregulated_df.count(axis=1).sort_values(ascending=False))
upregulated_count_df.columns = ['number']
upregulated_count_df['subsystem'] = list(upregulated_count_df.index.map(ijo_sub))
upregulated_count_df['mean'] = (significantly_upregulated_df.abs().mean(axis=1))
upregulated_count_df['std'] = (significantly_upregulated_df.abs().std(axis=1))
upregulated_count_df = upregulated_count_df.sort_values(by=['number'],ascending=False)[['number','subsystem','mean','std']]

downregulated_count_df = pd.DataFrame(significantly_downregulated_df.count(axis=1).sort_values(ascending=False))
downregulated_count_df.columns = ['number']
downregulated_count_df['subsystem'] = list(downregulated_count_df.index.map(ijo_sub))
downregulated_count_df['mean'] = (significantly_downregulated_df.abs().mean(axis=1))
downregulated_count_df['std'] = (significantly_downregulated_df.abs().std(axis=1))
downregulated_count_df = downregulated_count_df.sort_values(by=['number'],ascending=False)[['number','subsystem','mean','std']]


In [8]:
colors = list((cl.interp(cl.scales['11']['qual']['Set3'],11)))

flux_df = [on_df, off_df, significantly_upregulated_df, significantly_downregulated_df]
fig = subplots.make_subplots(rows=1, cols=4,print_grid=True, horizontal_spacing=0.05, 
                             specs=[[{"secondary_y": True}, {"secondary_y": True}, {"secondary_y": True}, 
                                     {"secondary_y": True}]],
                             subplot_titles=['On', 'Off', 'Upregulated', 'Downregulated'])

for i,temp_df in enumerate([on_count_df, off_count_df, upregulated_count_df, downregulated_count_df]):    
    for subsystem in temp_df[:10].subsystem.unique():
        reactions = list(temp_df[:10][temp_df[:10]['subsystem']==subsystem].index)
        if reactions:
            fig.add_trace(go.Bar(x=temp_df.loc[reactions].index, y=temp_df.loc[reactions].number,
                                 name=subsystem, marker_color=colors[subsystems.index(subsystem)],
                                 legendgroup=subsystem, showlegend=True, orientation='v', width=0.9, opacity=1), 
                          row=1, col=i+1, secondary_y=False)

            for reaction in reactions:
                temp = flux_df[i].loc[reaction].abs()[flux_df[i].loc[reaction].notna()].values
                temp[temp>15] = 15
                fig.add_trace(go.Scatter(x=[reaction]*len(flux_df[i].loc[reaction][flux_df[i].loc[reaction].notna()]),
                                         y=temp,
                                         name='Flux', mode='markers', legendgroup='Flux', showlegend=False,
                                         marker_color='black', marker_size=3, marker_opacity=0.25), 
                              row=1, col=i+1, secondary_y=True)
                
        fig.update_yaxes(range=(0, 60), secondary_y=False, row=1, col=i+1)
        fig.update_yaxes(range=(0, 15), tickvals=[0,5,10,15], secondary_y=True, row=1, col=i+1)




fig.update_layout(width=1050, height=350, showlegend=False)

fig.update_xaxes(showline=True, mirror=True, linewidth=1.5, linecolor='black', ticks='outside',
                 tickfont=dict(size=12, family='Arial', color='black'), tickangle=-90, range=(-0.5,9.5))

fig.update_yaxes(showline=True, mirror=True, linewidth=1.5, linecolor='black', ticks='outside',
                 tickfont=dict(size=12, family='Arial'), color='black', gridcolor='black', ticklen=2,
                 secondary_y=False)

fig.update_yaxes(showline=True, mirror=True, linewidth=1.5, linecolor='black', ticks='outside',
                 tickfont=dict(size=12, family='Arial'), color='black',gridcolor='black', ticklen=2,
                 secondary_y=True, titlefont=dict(family='Arial', size=18, color='black'),
                 side='right')
fig.update_yaxes(title='Number of Products', title_standoff=0, secondary_y=False, row=1, col=1 )
fig.update_yaxes(title='Change in Flux(mmol/gdw.h)', title_standoff=0, secondary_y=True, row=1, col=4 )


fig.show()
if not os.path.exists('images'):
    os.mkdir('images')
pio.write_image(fig, 'images/no_products_reactionwise_objA.svg')

This is the format of your plot grid:
[ (1,1) x,y,y2   ]  [ (1,2) x2,y3,y4 ]  [ (1,3) x3,y5,y6 ]  [ (1,4) x4,y7,y8 ]



In [41]:
expression_change_df = deepcopy(np.round(change_df.drop(['BIOMASS_Ec_iJO1366_core_53p95M','ATPM']),2)*100)
expression_change_df = expression_change_df-100
expression_change_df = expression_change_df[reversed(metabolite_order)]

expression_change_df['cov'] = expression_change_df.std(axis=1)/expression_change_df.mean(axis=1).abs()
expression_change_df['subsystem'] = expression_change_df.index.map(ijo_sub)
expression_change_df = expression_change_df.sort_values(by=['subsystem','cov'], ascending=[True, False])

In [42]:
(expression_change_df)

,Xanthosine,Uridine,Thymidine,Spermidine,Quinate,L-Tyrosine,L-Tryptophan,L-Phenylalanine,Inosine,Indole,Glutathione,Cytidine,Adenosine,L-Lysine,L-Leucine,L-Isoleucine,L-Idonic Acid,L-Histidine,L-Arginine,Hexanoate,D-Gluconic Acid,Citric Acid,5-Methylthioribose,5-Ketogluconate,α-Ketoglutarate,Xanthine,Thymine,Ornithine,O-Acetyl-L-serine,L-Valine,L-Proline,L-Glutamate,Hypoxanthine,Guanine,Cadaverine,Agmatine,Adenine,γ-Aminobutyrate,Uracil,Succinate,Putrescine,L-Threonine,L-Malate,L-Homoserine,L-Aspartate,L-Asparagine,Allantoin,Pyruvate,L-Serine,L-Lactate,L-Cysteine,L-Alanine,Glycerol3-phosphate,Glycerol,Glyceraldehyde,Dihydroxyacetone,D-Lactate,D-Alanine,3-Hydroxypropanoate,(S)-Propanediol,(R)-Propanediol,(R)-Glycerate,Glycolate,Glycine,Ethanolamine,Ethanol,Acetate,Acetaldehyde,Urea,Formate,cov,subsystem
G3PD2,-66.0,-66.0,-67.0,-68.0,-66.0,-66.0,-67.0,-66.0,-66.0,-65.0,-66.0,-66.0,-66.0,-65.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-65.0,-67.0,-66.0,-65.0,-66.0,-67.0,-66.0,-65.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-65.0,-66.0,-65.0,-67.0,2979.0,4037.0,4522.0,-66.0,-66.0,-66.0,-68.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-67.0,-66.0,-65.0,-67.0,-65.0,7.959581,Alternate Carbon Metabolism
DHAPT,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,377.0,-100.0,-23.0,-100.0,-100.0,353.0,-77.0,-100.0,-100.0,-59.0,-100.0,-100.0,-100.0,-100.0,-100.0,227.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,403.0,-100.0,-100.0,-100.0,227.0,-100.0,-100.0,-100.0,-100.0,338.0,-83.0,-100.0,-100.0,-100.0,-100.0,8.0,-100.0,-100.0,333.0,8.0,408.0,-95.0,-100.0,3.112127,Alternate Carbon Metabolism
A5PISO,-66.0,-66.0,-67.0,-68.0,-66.0,-66.0,-67.0,-66.0,-66.0,-65.0,-66.0,-66.0,-66.0,-65.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-65.0,-67.0,-66.0,-65.0,-66.0,-67.0,-66.0,-65.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-65.0,-66.0,-65.0,-67.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-68.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-67.0,-66.0,-65.0,-67.0,-65.0,0.009646,Alternate Carbon Metabolism
PSP_L,123.0,-66.0,-11.0,18.0,-66.0,-66.0,-67.0,-66.0,115.0,-65.0,126.0,-66.0,109.0,-65.0,-66.0,-66.0,-66.0,27.0,-66.0,-67.0,-66.0,-67.0,21.0,-66.0,-100.0,340.0,48.0,-66.0,201.0,-67.0,-68.0,-78.0,304.0,295.0,-65.0,-66.0,280.0,-82.0,-66.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,394.0,-69.0,396.0,-65.0,156.0,-68.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-68.0,-65.0,-65.0,-100.0,-100.0,464.0,236.0,-67.0,-100.0,-66.0,-100.0,-65.0,133.447237,Amino Acid Metabolism
PSERT,123.0,-66.0,-11.0,18.0,-66.0,-66.0,-67.0,-66.0,115.0,-65.0,126.0,-66.0,109.0,-65.0,-66.0,-66.0,-66.0,27.0,-66.0,-67.0,-66.0,-67.0,21.0,-66.0,-100.0,340.0,48.0,-66.0,201.0,-67.0,-68.0,-78.0,304.0,295.0,-65.0,-66.0,280.0,-82.0,-66.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,394.0,-69.0,396.0,-65.0,156.0,-68.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-68.0,-65.0,-65.0,-100.0,-100.0,464.0,236.0,-67.0,-100.0,-66.0,-100.0,-65.0,133.447237,Amino Acid Metabolism
PGCD,123.0,-66.0,-11.0,18.0,-66.0,-66.0,-67.0,-66.0,115.0,-65.0,126.0,-66.0,109.0,-65.0,-66.0,-66.0,-66.0,27.0,-66.0,-67.0,-66.0,-67.0,21.0,-66.0,-100.0,340.0,48.0,-66.0,201.0,-67.0,-68.0,-78.0,304.0,295.0,-65.0,-66.0,280.0,-82.0,-66.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,394.0,-69.0,396.0,-65.0,156.0,-68.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-68.0,-65.0,-65.0,-100.0,-100.0,464.0,236.0,-67.0,-100.0,-66.0,-100.0,-65.0,133.447237,Amino Acid Metabolism
SERAT,-66.0,-66.0,-67.0,329.0,-66.0,-66.0,-67.0,-66.0,-66.0,-65.0,602.0,-66.0,-66.0,-65.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,339.0,-66.0,-66.0,-65.0,-67.0,-66.0,1788.0,-66.0,-67.0,-66.0,-65.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-66.0,-65.0,-66.0,1469.0,-67.0,-66.0,-65.0,-66.0,-66.0,-66.0,-66.0,-68.0,-

In [43]:
small_subsystems = expression_change_df.groupby('subsystem')['subsystem'].count()[expression_change_df.groupby('subsystem')['subsystem'].count()<=5].index
display(expression_change_df.groupby('subsystem')['subsystem'].count().sort_values(ascending=False))
expression_change_df = expression_change_df.drop(expression_change_df[expression_change_df['subsystem'].isin(small_subsystems)].index)
expression_change_df = expression_change_df.drop(expression_change_df[expression_change_df['subsystem'].isin(['Lipopolysaccharide Biosynthesis / Recycling', 'Glycerophospholipid Metabolism', 'Cofactor and Prosthetic Group Biosynthesis', 'Cofactor and Prosthetic Group Biosynthesis', 'Oxidative Phosphorylation'])].index)
expression_change_df = expression_change_df.drop('cov',axis=1)

subsystem
Amino Acid Metabolism                          88
Membrane Lipid Metabolism                      31
Cell Envelope Biosynthesis                     23
Purine and Pyrimidine Biosynthesis             21
Nucleotide Salvage Pathway                     15
Lipopolysaccharide Biosynthesis / Recycling    12
Glycolysis/Gluconeogenesis                     10
Glycerophospholipid Metabolism                 10
Pentose Phosphate Pathway                       9
Citric Acid Cycle                               8
Cofactor and Prosthetic Group Biosynthesis      7
Oxidative Phosphorylation                       6
Folate Metabolism                               4
Alternate Carbon Metabolism                     3
Pyruvate Metabolism                             3
Murein Biosynthesis                             2
Unassigned                                      2
Anaplerotic Reactions                           1
Name: subsystem, dtype: int64

In [44]:
expression_change_df['subsystem'].values
subsystem_labels = []
subsystem_set = []
for subsystem in expression_change_df['subsystem'].values:
    if subsystem in subsystem_set:
        subsystem_labels.append('')
    else:
        subsystem_labels.append(subsystem)
        subsystem_set.append(subsystem)

In [45]:

layout = go.Layout(height=1000,
                   width=800,
                   margin=dict(l=400),
                   xaxis=dict(#title='Product',
                              #titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True,
                              mirror=True,
                              ticks='outside',
                              ticklen=0,
                              nticks=1,
                              tickfont=dict(size=10, color='black'),
                              ticktext=subsystem_labels,
                              tickvals=expression_change_df.index,
                              tickangle=-90),
                   yaxis=dict(#title='Product',
                              #titlefont=dict(family='Arial', size=18, color='black'),
                              showline=True,
                              mirror=True,
                              ticks='outside',
                              ticklen=3,
                              tickfont=dict(size=9.5, color='black')
                   
)
                   
             )
fig = go.Figure(data=[go.Heatmap(z=expression_change_df[expression_change_df.columns[:-1]].values.transpose(),
                                y=expression_change_df.columns[:-1],
                                x=expression_change_df.index,
                                zmin=-100, zmax=100, colorscale=list(reversed(pycolors.diverging.RdBu[1:-1])),
                                colorbar=dict(thickness=20, len=0.5,ticks='outside',
                                              tickfont=dict(size=12,color='black'),
                                              nticks=20))],
                layout=layout)


plot(fig)

pio.write_image(fig, 'images/flux_change_heatmap_objA_2.svg')